## Анализ топа фильмов

<img src='top_movies.jpg' style='width:600px;height:400px'/>

----------
Комания "Тралала" была известна не только по анализу игр, но так же и по анализу лучших фильмов мирового кинематографа, поэтому ей было поручено:

1. Успешно загрузить данные, подготовить их и если потребуется - преобразовать для дальнейшего исследования. 

2. Вычислить статистические признаки по жанрам. 

3. Вычислить статистические признаки по режессерам.

4. А так же провести анализ по возрастному рейтингу и понять существует ли зависимость между возрастным рейтингом и доходом фильмов.

----------

## Загрузка данных и подготовка к анализу

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import warnings

warnings.filterwarnings("ignore")
sns.set_style('darkgrid')

In [ ]:
try:
    df_imdb = pd.read_csv('Top_250_Movies.csv', index_col=[0])
except:
    print('Ошибка, проверьте правильность заполненных данных')
else:
    print('Данные успешно загружены')


----------
Общая информация по датасету:




In [ ]:
df_imdb.head()

--------
Информация о пропусках и типах данных

---------

In [ ]:
df_imdb.info()

-------------------------------------

df_imdb — результат запроса. В нём содержится следующая информация:

- `name` - Название фильма
- `year` - Год выхода фильма
- `rating` - Оценка фильма
- `genre` - Жанр фильма
- `certificate` - Возрастной рейтинг фильма
- `run_time` - Продолжительность фильма 
- `tagline` - Слоган
- `budget` - Бюджет
- `box_office` - Общие кассовые сборы по всему миру
- `casts` -  Все актеры фильма
- `directors` - Режиссер фильма
- `writers` - Сценарист фильма

Пропуски в таблицах не наблюдаются. Необходимости преобразовывать типы нет.

Для колонок `year`, `rating` можем уменьшить занимаемую память

-------------------------------------

In [ ]:
df_imdb = df_imdb.astype({'year': 'int16', 
                          'rating': 'float16'})

In [ ]:
df_imdb.info()

----------
Немного сэкономили занимаемую память

----------

----------
#### Наличие дубликатов в датасете:

In [ ]:
df_imdb.duplicated().sum()

Дубликаты не обнаружены, можем перейти к анализу.

----------

## Проведем анализ с жанрами: 

----------
##### Средние рейтинги по жанрам:

----------


In [ ]:
df_imdb.groupby(by='genre', as_index=False).mean()[['genre', 'rating']].nlargest(10, 'rating')

----------

##### Средняя касса по жанрам:

----------

In [ ]:
df_genre_bo_mean = df_imdb[['name', 'genre']]
df_genre_bo_mean['box_office'] = df_imdb[df_imdb['box_office'].str.isnumeric()]['box_office'].apply(int)

df_genre_bo_mean = df_genre_bo_mean.groupby(by='genre', as_index=False).mean()[['genre', 'box_office']].nlargest(10, 'box_office')
df_genre_bo_mean


Как мы можем наблюдать, только у одной строки с жанрами средний доход выходит больше чем на миллиард. Другие жанры с учетом того, что мы взяли десять самых кассовых, находятся, не считая еще группы жанров, заметно ниже. И позволяет нам предположиться, что потребитель выбирает в среднем чаще фильмы или мульфильмы с приключениями, экшном, драмой, фантастикой или фэнтази, чем остальные типы жанров

Создадим диаграмму, которая визуально поделит нам на три части среднюю кассу.



In [ ]:
plt.figure(figsize=(10, 4), dpi=200)
plt.title('Cредняя касса по жанрам в миллиардах')
plt.xticks(rotation=30)

sns.histplot(data=df_genre_bo_mean, x='box_office', y='genre', bins=3);

----------

##### Найдем к каким жанрам относятся 10 самых лучших фильмов.

----------

In [ ]:
top_10 = df_imdb.nlargest(10, 'rating')

plt.figure(figsize=(10, 4), dpi=200)
plt.title('Топ-10 фильмов по жанру')
plt.xticks(rotation=30)

sns.countplot(data=top_10, x='genre', palette='magma');



На таблице можно заметить, что 10-ка наиболее оценненых фильмов принадлежит столбцу с жанрами - 'Crime, Drama', на втором месте - 'Action, Adventure, Drama'. Остальные фильмы распределились равномерно между собой.

----------

## Проведем анализ по режиссерам:

----------
##### Средний доход фильмов по режиссерам

----------

In [ ]:
df_dir_mean = df_imdb[['name', 'directors']]
df_dir_mean['box_office'] = df_imdb[df_imdb['box_office'].str.isnumeric()]['box_office'].apply(int)

df_dir_mean = df_dir_mean.groupby(by='directors', as_index=False).mean()[['directors', 'box_office']].nlargest(10, 'box_office')

df_dir_mean


На результате выдим, что у большинства, а имеенно у 6-ти режиссеров, киноленты собрали в среднем миллиард долларов.

----------

----------
##### Режиссеры, которые попали в рейтинг чаще остальных.

----------

In [ ]:
df_dir_count = df_imdb[['name', 'directors']]
df_dir_count.groupby(by='directors').count().nlargest(10, 'name')


Как мы можем наблюдать по выводу - Akira Kurosawa, Christopher Nolan, Martin Scorsese, Stanley Kubrick и Steven Spielberg удостоились чести попасть в рейтинг по 7 раз каждый.

----------

----------
##### Режиссеры 10-ти самых высокооцененных фильмов:

----------

In [ ]:
top_10_d = df_imdb.nlargest(10, 'rating')

plt.figure(figsize=(10, 5), dpi=200)
plt.title('Режиссеры десяти самых высокооцененных фильмов')
plt.xticks(rotation=30)
sns.countplot(data=top_10_d, x='directors', palette='magma');


Судя по выводу диаграммы,  два режиссера, а именно 'Francis Ford Coppola' и 'Peter Jackson' являются заслуженными иконами киноиндустрии, ведь сразу несколько их фильмов попали в топ-10 самых высокоценных фильмов.

----------

----------
##### Давайте взглянем получше на эти фильмы.

----------

In [ ]:
df_imdb[['name', 'genre', 'directors', 'rating']].nlargest(10, 'rating')

----------

##### Найдем какую общую сумму заработали все фильмы режессеров и выберем 10 наибольших.

----------

In [ ]:
df_box_of_with_directors = df_imdb[['name', 'directors']]
df_box_of_with_directors['box_office'] = df_imdb[df_imdb['box_office'].str.isnumeric()]['box_office'].apply(int)

directors_box_office = df_box_of_with_directors.groupby(by='directors').sum().nlargest(10, 'box_office')


plt.figure(figsize=(10, 5), dpi=200)
plt.title('Общий доход фильмов по каждому режессеру в миллиардах')
plt.xticks(rotation=30)

sns.histplot(data=directors_box_office, x='box_office', y='directors', bins=3);

In [ ]:
directors_box_office 


На графике и в выводе результата мы можем наблюдать что:

Режиссеры - Jon Watt, Joseph Kosinski, David Yates, Martin Scorsese, Quentin Tarantino и Todd Phillips
принесли кассу в промежутке, который начинается с чуть более 1-го миллиарда и заканчиваясь почти на 2-ух миллиардах.

Режиссеры - Peter Jackson, Steven Spielberg принесли кассу в промежутку с почти 3-ех до 3.25 миллирадов.

Режиссеры - Christopher Nolan и братья Anthony Russo, Joe Russo сильно вырываются вперед и принесли кассу в промежутке с 4.2 до 4.8 миллиардов соответственно.

----------

## Проведем анализ по возрастному рейтингу:

----------
##### Посмотрим на количество фильмов по рейтингу.

----------

In [ ]:
plt.figure(figsize=(10, 5), dpi=200)
plt.title('Количество фильмов по возрастному рейтингу')
plt.xticks(rotation=30)

sns.countplot(data=df_imdb, x='certificate');



Из диаграммы мы наблюдаем, что большинство фильмов принадлежат возрастному рейтингу R, который предпологает много насилия и сквернословия. Даллее будут расмотрены обозначения для рейтингов.

Вот что означают эти возрастные рейтинги:
    
    R: Этот сертификат означает "Действительно, действительно не для детей". Ожидайте много насилия и сквернословия.
    
    PG: Это означает "Вполне подходит для детей". Могут присутствовать легкие выражения или насилие, но в целом фильм безопасен для маленьких глаз.
    
    PG-13: "Родителям настоятельно рекомендуется брать с собой подростков". Этот фильм для детей постарше, которые могут выдержать немного больше действий и интенсивности.
    
    Not Rated: Это код для "Мы понятия не имеем, что в этом фильме". Это дикая карта, так что пристегнитесь и надейтесь на лучшее.
    
    G: "Хорошо для всех". Этот фильм безопасен даже для самых маленьких зрителей. Ожидайте много поющих животных и счастливых концовок.
    
    Passed: Этот знак - старый. Это означает, что фильм был одобрен цензорами, которые посчитали его "Допустимым для публичного просмотра".
    
    Approved: Еще один старый, но добрый фильм. Это означает, что цензоры сочли фильм "Приемлемым для всех зрителей".
    
    18+: Этот фильм не для слабонервных. Это как фильм с рейтингом R, но еще более напряженный. Оставьте детей дома и приготовьтесь прикрыть глаза.
    
    Not Available: Этот сертификат означает "Вам придется посмотреть его и узнать". Это тайна!
    
    TV-PG: "Довольно хорошо для телевидения". Это как сертификат PG для фильмов, но для телепередач. Идеально подходит для семейных ситкомов.
    
    Unrated: Это означает, что фильм еще не прошел цензуру. Смотрите на свой страх и риск!
    
    X: Это код для "XXX-rated". Ожидайте много секса и наготы. Определенно не для первого свидания.
    
    13+: Это для подростков и тинэйджеров. Это как PG-13, но для телевизионных шоу.
    
    TV-MA: "Только для зрелой аудитории". Это как фильм с рейтингом R, но для телешоу. Ожидайте много насилия, секса и нецензурной лексики.
    
    GP: Это означает "Для широкой аудитории, рекомендуется родительское руководство". Это как PG, но родители должны быть немного более вовлечены в процесс принятия решения о том, подходит ли это для их детей.
    
----------

----------
##### Рассмотрим как рейтинг фильмов влияет на доход.

----------

In [ ]:
df_rat_bo = df_imdb[['name', 'certificate']]
df_rat_bo['box_office'] = df_imdb[df_imdb['box_office'].str.isnumeric()]['box_office'].apply(int)

plt.figure(figsize=(10, 5), dpi=200)
plt.title('Доход фильмов в миллиардах')
plt.xticks(rotation=30)

sns.scatterplot(data=df_rat_bo[df_rat_bo['box_office'].notna()], x='certificate', y='box_office');




На графике можно увидеть, что доход свыше полумиллиарда, принадлежит примущественно фильмам с семейным рейтингом, на который можно взять детей и не бояться, что они смогут увидеть что-либо запрещенное. Тогда как фильмы с рейтинком R, собирает в основном кассу до полумиллиарда и изредка больше.


Для того, чтобы убедиться в том, что семейные фильмы действительно приносят больше дохода, чем остальные. Найдем среднее по каждому из рейтингов.



In [ ]:
df_rat_bo[df_rat_bo['box_office'].notna()].\
                    groupby(by='certificate').mean().\
                    sort_values(by='box_office', ascending=False)



Как мы видим по выводу, фильмы с возрастной категорией для детей и подростков - PG-13, G и PG действительно зарабатывают денег больше чем остальные фильмы в других категориях.

----------

----------
Так же наблюдается заметный выброс кассы в возрастном рейтиге 'PG-13', давайте взглянем на этот фильм.

In [ ]:
df_rat_bo.nlargest(1, 'box_office')

В результате можем видеть фильм Avengers: Endgame вышедший в 2019 году, который по используемым данным на тот момент собрал кассу в 2.8 миллиарда долларов. 

----------

## Вывод:

- Из csv-файла извлекли необходмые данные - пропуски и дубликаты не были обнаружены.


- Были проведены иследование по жанрам:

     - Средние значения рейтинга по жанрам. Жанры, которые имеют в среднем рейтинг выше других - Adventure, Western и Action, Adventure, Drama
     - Средние значения сборов кассы по жанрам. Жанры, которые в среднем зарабатывают больше всех - Action, Adventure, Drama
     - К каким жанрам относятся 10 самых высокооцененных фильмов. Жанры - Crime, Drama чаще других находится в рейтинге 10 самых оценненых фильмов.


- Был проведен анализ по режиссерам:

     - Рассчитан средний доход фильма по каждому режиссеру. Выше всех средний доход по фильмам принадлежит братьям Anthony Russo и Joe Russo.
     
     - Были найдены режиссеры, чаще всего попадающие в этот топ. Akira Kurosawa, Christopher Nolan, Martin Scorsese, Stanley Kubrick и Steven Spielberg удостоились чести попасть в рейтинг по 7 раз каждый.

     - Найдены режиссеры 10-ти самых высокооцененных фильмов. Примечательно, что Francis Ford Coppola и Peter Jackson дважды вошли в этот список.
     
     - Был посчитан общий доход фильмов по каждому режиссеру в миллиардах. Больше всех принесли доход фильмы братьев Anthony Russo и Joe Russo.
     

- Был проведен анализ по возрастному рейтингу:

     - Больше всего фильмов находившихся в этом рейтинге, приходится на возрастной рейтинг R - Этот сертификат означает "Действительно, действительно не для детей". Ожидайте много насилия и сквернословия.

     - Было рассмотрено как рейтинг влияет на доход. Чаще всего фильмы с семейным рейтигном, куда родители могут взять детей зарабатывают больше полумиллирда долларов. И в среднем зарабатывают больше остальных.

     - Самым кассовым фильмом в топе является фильм Avengers: Endgame вышедший в 2019 году, который заработал 2.8 миллиарда долларов и у которого рейтинг PG-13: "Родителям настоятельно рекомендуется брать с собой подростков". Этот фильм для детей постарше, которые могут выдержать немного больше действий и интенсивности.